In [1]:
import os
import requests
import re
from bs4 import BeautifulSoup
import openpyxl

In [2]:
def hasdigit(url):
    return any(char.isdigit() for char in url)

In [3]:
def HavaAlphabet(url):
    return any(char.isalpha() for char in url)

In [4]:
def bbc_business_Filter(url):
    feature1 = url[-17:-9]
    feature2 = url[-8:]
    if feature1 != 'business':
#         print(url)
        return False
    elif HavaAlphabet(feature2):
        return False
    else:
        return True

In [5]:
bbc_business = 'https://www.bbc.com/news/business'

In [6]:
saving_path = '.\\7_websites_output'
if not os.path.exists(saving_path):
    os.makedirs(saving_path)

In [7]:
BBC_homePage = "#topos-component"
BBC_NewsPage = ".ssrcss-rgov1k-MainColumn.e1sbfw0p0"

In [8]:
def ft_BBC():
    prefix = 'https://www.bbc.com'
    r = requests.get(bbc_business)

    if r.status_code == 200:  # print(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        result = soup.select(BBC_homePage)

    raw_url_list = []
    target_url_list = []
    for results in result:
        class_a = results.find_all("a")

    for i in class_a:
        if i['href'][0] == '/':
            raw_url_list.append(str(prefix+i['href']))
        else:
            raw_url_list.append(str(i['href']))

    for url in raw_url_list:
        if bbc_business_Filter(url):
            if url in target_url_list:
                pass
            else:
                target_url_list.append(url)

    workbook = openpyxl.Workbook()
    sheet = workbook.active

    ctr = 1

    for url in target_url_list[:]:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            result = soup.select(BBC_NewsPage)
            title_select = soup.find("h1")
            sheet['A'+str(ctr)] = title_select.text
            # print("----------------------------------------------------------------------------")
            # print(title_select.text)
            # print(url)

            sheet['B'+str(ctr)] = url

            content = []
            for results in result:
                paragraph = results.find_all("p")

                for sentence in paragraph:
                    content.append(sentence.text)

            if content == []:
                print("Can't crawler the web")
                print(url)
                sheet['C'+str(ctr)] = "Can't Crawler the Web"
                sheet['D'+str(ctr)] = ''
                ctr += 1

            else:
                for sentence in content:
                    sheet['D'+str(ctr)] = str(sentence)
    #                 print(sentence)
                    ctr += 1

    workbook.save(saving_path+'\\BBC.xlsx')
    
    print('finish')

In [9]:
ft_BBC()

finish
